In [1]:
import sys
from collections.abc import Iterable
import pickle

def get_size(obj, seen=None):
    """Recursively find the size of objects in bytes."""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0

    # Mark as seen
    seen.add(obj_id)

    # If the object is a dictionary, iterate over its items
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    # If the object is an iterable (list, tuple, set, etc.), iterate over its elements
    elif isinstance(obj, Iterable) and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    
    return size

def convert_bytes_to_gb(bytes_size):
    return bytes_size / (1024 ** 3)

obj = {
    'a': [1, 2, 3, 4, 5],
    'b': {'x': 10, 'y': 20},
    'c': (3.14, 2.71, 1.62),
    'd': 'string',
    'e': b'bytes'
}

In [2]:
INPUT_DIR = '/home/azureuser/cloudfiles/code/Users/e1310988/satellite-te/input/starlink'

In [3]:
import numpy as np

def convert_array_to_dict(example_array):
    indices = []
    values = []

    for index in np.ndindex(example_array.shape):
        value = example_array[index]
        if value != 0:
            indices.append(index)
            values.append(value)

    result_dict = {'size': example_array.shape[0],'edge_list': np.array(indices), 'weight_list': np.array(values)}

    return result_dict

def convert_dict_to_array(example_dict):

    result_array = np.zeros((example_dict['size'], example_dict['size']), dtype=np.float32)

    for index, value in zip(example_dict['edge_list'], example_dict['weight_list']):
        result_array[index] = value

    return result_array

In [4]:
import networkx as nx

with open(f'{INPUT_DIR}/../iridium/IridiumDataSet14day20sec_Int15/topo_0/paths_num-5_edge-disjoint-False_dist-metric-min-hop-dict.pkl', 'rb') as file:
    iridium_path = pickle.load(file)

iridium_topo = nx.read_gpickle(f'{INPUT_DIR}/../iridium/IridiumDataSet14day20sec_Int15/topo_0/graph.gpickle')

iridium_tm_train_matrix = []
iridium_tm_train_list = []

with open(f'{INPUT_DIR}/../iridium/IridiumDataSet14day20sec_Int15/topo_1/tm_train.pkl', 'rb') as file:
    iridium_tm_train_full = pickle.load(file)

for i in range(1, 51):
    iridium_tm_train_matrix.append(iridium_tm_train_full[i])
    iridium_tm_train_list.append(convert_array_to_dict(iridium_tm_train_full[i]))

In [5]:
import numpy as np

def non_zero_count(lst):
    return len(list(filter(lambda x: x != 0, lst)))

tm_num = 0

for tm in iridium_tm_train_matrix:
    tm_num += np.count_nonzero(tm)


print(tm_num/len(iridium_tm_train_matrix))

print(len(next(iter(iridium_path.values()))) * len(iridium_path))

135.42
86460


In [4]:
import os
import sys
import random
import networkx as nx

sys.path.append(os.path.join(os.getcwd(), ".."))

from lib.data.starlink.orbit_params import OrbitParams
from lib.data.starlink.ism import InterShellMode as ISM
from lib.data.starlink.user_node import generate_sat2user


def construct_from_edge(edge_list, params):
    """Construct a networkx graph from a list of edges."""

    sat2user = generate_sat2user(params.Offset5, params.GrdStationNum, params.ism)
    G = nx.DiGraph()
    G.add_nodes_from(range(params.graph_node_num))
    ## 1. Inter-satellite links
    for e in edge_list:
        if (e[0] == e[1]) :
            continue
        G.add_edge(e[0], e[1], capacity=params.isl_cap)
        # G.add_edge(e[1], e[0], capacity=params.isl_cap)
    ## 2. User-satellite links
    for i in range(params.Offset5):
        # Uplink
        G.add_edge(sat2user(i), i, capacity=params.uplink_cap)
        # Downlink
        G.add_edge(i, sat2user(i), capacity=params.downlink_cap)
    ## 3. Inter ground station links
    for i in range(params.GrdStationNum):
        for j in range(params.GrdStationNum):
            if i == j:
                continue
            G.add_edge(i + params.Offset5, j + params.Offset5, capacity=0)
            G.add_edge(j + params.Offset5, i + params.Offset5, capacity=0)
    
    return G

In [5]:
with open(f'{INPUT_DIR}/starlink_500/GrdStation_teal/topo_0/paths_num-5_edge-disjoint-False_dist-metric-min-hop-dict.pkl', 'rb') as file:
    starlink500_path = pickle.load(file)

reduced = 8

params = OrbitParams(
    GrdStationNum=222,
    Offset5=round(2 * 22 * 72 / reduced),
    graph_node_num=round(2 * 22 * 72 / reduced) * 2 + 222,
    isl_cap=200,
    uplink_cap=800,
    downlink_cap=800,
    ism=ISM.GRD_STATION,
)

with open(f'{INPUT_DIR}/starlink_500/GrdStation_teal/topo_0/graph_edge.pickle', 'rb') as file:
    starlink500_topo = construct_from_edge(pickle.load(file), params)

starlink500_tm_train_list = []
starlink500_tm_train_matrix = []

for i in range(1, 51):
    with open(f'{INPUT_DIR}/starlink_500/GrdStation_teal/topo_0/tm_train/{i}.pkl', 'rb') as file:
        tm = pickle.load(file)
        starlink500_tm_train_list.append(tm)
        starlink500_tm_train_matrix.append(convert_dict_to_array(tm))

In [ ]:
def non_zero_count(lst):
    return len(list(filter(lambda x: x != 0, lst)))

tm_num = 0

for tm in starlink500_tm_train_list:
    tm_num += non_zero_count(tm['weight_list'])

print(tm_num/len(starlink500_tm_train_list))

print(len(next(iter(starlink500_path.values()))) * len(starlink500_path))

In [15]:
with open(f'{INPUT_DIR}/starlink_1500/GrdStation_teal/topo_0/paths_num-5_edge-disjoint-False_dist-metric-min-hop-dict.pkl', 'rb') as file:
    starlink1500_path = pickle.load(file)

reduced = 2

params = OrbitParams(
    GrdStationNum=222,
    Offset5=round(2 * 22 * 72 / reduced),
    graph_node_num= round(2 * 22 * 72 / reduced) * 2+ 222,
    isl_cap=200,
    uplink_cap=800,
    downlink_cap=800,
    ism=ISM.GRD_STATION,
)

with open(f'{INPUT_DIR}/starlink_1500/GrdStation_teal/topo_0/graph_edge.pickle', 'rb') as file:
    starlink1500_topo = construct_from_edge(pickle.load(file), params)

starlink1500_tm_train_list = []
starlink1500_tm_train_matrix = []

for i in range(1, 51):
    with open(f'{INPUT_DIR}/starlink_1500/GrdStation_teal/topo_0/tm_train/{i}.pkl', 'rb') as file:
        tm = pickle.load(file)
        starlink1500_tm_train_list.append(tm)
        starlink1500_tm_train_matrix.append(convert_dict_to_array(tm))

In [ ]:
def non_zero_count(lst):
    return len(list(filter(lambda x: x != 0, lst)))

tm_num = 0

for tm in starlink1500_tm_train_list:
    tm_num += non_zero_count(tm['weight_list'])

print(tm_num/len(starlink1500_tm_train_list))

print(len(next(iter(starlink1500_path.values()))) * len(starlink1500_path))

In [5]:
def from_5000_dict_to_tm_list(tm_dict, size):    
    # Extract indices and values
    indices = []
    values = []

    for key, value in tm_dict.items():
        index = tuple(map(int, key.split(',')))  # Split the string key and convert to tuple of ints
        indices.append(index)
        values.append(value)

    # Convert lists to NumPy arrays
    
    indices_array = np.array(indices)
    values_array = np.array(values)

    # Construct the result dictionary
    result_dict = {'size': size, 'edge_list': indices_array, 'weight_list': values_array}

    return result_dict

In [6]:
params = OrbitParams(
    GrdStationNum=222,
    Offset5=4236,
    graph_node_num=8694,
    isl_cap=200,
    uplink_cap=800,
    downlink_cap=800,
    ism=ISM.GRD_STATION,
)

import pickle

with open(f'{INPUT_DIR}/DataSetForSaTE100/GrdStation/StarLink_DataSetForAgent100_5000_B.pkl', 'rb') as file:
    starlink5000_data = pickle.load(file)

starlink5000_path = starlink5000_data[0]['path']
starlink5000_topo = construct_from_edge(starlink5000_data[0]['graph'], params)

starlink5000_tm_train_list = []
starlink5000_tm_train_matrix = []

for i in range(1, 51):
    tm_list = from_5000_dict_to_tm_list(starlink5000_data[i]['tm'], params.graph_node_num)
    starlink5000_tm_train_list.append(tm_list)
    starlink5000_tm_train_matrix.append(convert_dict_to_array(tm_list))

In [ ]:
path_num = 0
tm_num = 0

for i in range(len(starlink5000_data)):
    path_num += len(starlink5000_data[i]['path'])
    tm_num += len(starlink5000_data[i]['tm'])

print(path_num/5000, tm_num/5000)

print(len(starlink5000_path['8343, 6455']))

In [7]:
def get_variable_name(obj, global_vars):
    matches = [name for name, value in global_vars.items() if value is obj]
    return matches[0] if matches else None

object_names = [
    'iridium_path', 'iridium_topo', 'iridium_tm_train_matrix', 'iridium_tm_train_list',
    'starlink500_path', 'starlink500_topo', 'starlink500_tm_train_matrix', 'starlink500_tm_train_list',
    'starlink1500_path', 'starlink1500_topo', 'starlink1500_tm_train_matrix', 'starlink1500_tm_train_list',
    'starlink5000_path', 'starlink5000_topo', 'starlink5000_tm_train_matrix', 'starlink5000_tm_train_list'
]

for name in object_names:
    if name in globals():
        obj = globals()[name]
        if not obj:
            continue
        print(f"Total size of the {name}: {convert_bytes_to_gb(get_size(obj))} GB")
    else:
        print(f"{name} is not defined, skipping...")


iridium_path is not defined, skipping...
iridium_topo is not defined, skipping...
iridium_tm_train_matrix is not defined, skipping...
iridium_tm_train_list is not defined, skipping...
starlink500_path is not defined, skipping...
starlink500_topo is not defined, skipping...
starlink500_tm_train_matrix is not defined, skipping...
starlink500_tm_train_list is not defined, skipping...
starlink1500_path is not defined, skipping...
starlink1500_topo is not defined, skipping...
starlink1500_tm_train_matrix is not defined, skipping...
starlink1500_tm_train_list is not defined, skipping...
Total size of the starlink5000_path: 0.015014640986919403 GB
Total size of the starlink5000_topo: 0.00022675469517707825 GB
Total size of the starlink5000_tm_train_matrix: 14.079381860792637 GB
Total size of the starlink5000_tm_train_list: 0.008303857408463955 GB


In [1]:
import pickle

with open('/data/projects/11003765/sate/input/starlink/starlink_528/GrdStation/StarLink_DataSetForAgent100_5000_Size528.pkl', 'rb') as file:
    data = pickle.load(file)


len(data)

100